# XAI Experiment Data Cleaning

In [1]:
import json
import pandas as pd
import requests
import io
from urllib.request import urlopen
import json
import logging
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pandas.io.json import json_normalize

In [2]:
# load JSON
def load_data(filename):
    df = pd.read_json(filename)
    data_sorted = df['formAnswers'].apply(pd.Series).join(df[['swymerId', 'workDurationInSeconds']])
    return data_sorted.fillna("")

In [3]:
# from google.colab import files

# uploaded = files.upload()

In [4]:
#load json files
v1_q = load_data('data/raw/answers-v1-basic-group-2-qualification.json')
v1_m = load_data('data/raw/answers-v1-basic-group-2-main.json')

v2_q_p1 = load_data('data/raw/answers-v2-salient-group-2-qualification.json')
v2_m_p1 = load_data('data/raw/answers-v2-salient-group-2-main.json')

v2_q_p2 = load_data('data/raw/answers-v2-salient-group-2-qualification-part2.json')
v2_m_p2 = load_data('data/raw/answers-v2-salient-group-2-main-part2.json')

v2_q = pd.concat([v2_q_p1, v2_q_p2])
v2_m = pd.concat([v2_m_p1, v2_m_p2])

v3_q = load_data('data/raw/answers-v3-explanations-group-2-qualification.json')
v3_m = load_data('data/raw/answers-v3-explanations-group-2-main.json')

In [5]:
journalists = load_data('data/raw/answers-journalists.json')

In [6]:
#confert to pandas data frame
df_v1_q = pd.DataFrame(v1_q)
df_v1_m = pd.DataFrame(v1_m)

df_v2_q = pd.DataFrame(v2_q)
df_v2_m = pd.DataFrame(v2_m)

df_v3_q = pd.DataFrame(v3_q)
df_v3_m = pd.DataFrame(v3_m)

df_j = pd.DataFrame(journalists)

In [7]:
# remove columns from df_XX_q which are redundant or not needed for the analysis
reduntant_columns = [
  "METADATA.FEATURE",
  "METADATA.GROUP",
  "confirm.0",
]

df_v1_q = df_v1_q.drop(columns=reduntant_columns)
df_v2_q = df_v2_q.drop(columns=reduntant_columns)
df_v3_q = df_v3_q.drop(columns=reduntant_columns)

# get a list of all column names starting with "tutorial."
tutorial_columns = [col for col in df_v1_q.columns if col.startswith("tutorial.")]
tutorial_columns.append("understand-task")

# remove tutorial columns from both dataframes
df_v1_q = df_v1_q.drop(columns=tutorial_columns)
df_v1_m = df_v1_m.drop(columns=tutorial_columns)

df_v2_q = df_v2_q.drop(columns=tutorial_columns)
df_v2_m = df_v2_m.drop(columns=tutorial_columns)

df_v3_q = df_v3_q.drop(columns=tutorial_columns)
df_v3_m = df_v3_m.drop(columns=tutorial_columns)

# remove "METADATA.PART" columns from both dataframes
df_v1_q = df_v1_q.drop(columns=["METADATA.PART"])
df_v1_m = df_v1_m.drop(columns=["METADATA.PART"])

df_v2_q = df_v2_q.drop(columns=["METADATA.PART"])
df_v2_m = df_v2_m.drop(columns=["METADATA.PART"])

df_v3_q = df_v3_q.drop(columns=["METADATA.PART"])
df_v3_m = df_v3_m.drop(columns=["METADATA.PART"])

In [8]:
df_j = df_j.drop(columns=["METADATA.PART", "confirm.0"] + tutorial_columns)

In [9]:
# merge df_v1_q and df_v1_m on swymerId, 
# only keep rows where aliasId is in both dataframes (inner join)
# keep all columns from both and suffix them with .qualification and .main (e.g. for POINTS)
df_v1 = pd.merge(df_v1_q, df_v1_m, on='swymerId', how='inner', suffixes=('.qualification', '.main'))
df_v1

,information-literacy.identify-misinformation,information-seeking.sources.online-newspaper,expectations.sceptical-about-ai,information-seeking.sources.personal-social-networks,personal-code,information-literacy.efficiently-use-subscribed-and-openaccess,information-seeking.purpose.keep-up-to-date,POINTS.qualification,information-seeking.truthfulness.facebook,information-seeking.purpose.general-awareness,...,ai-system-evaluation.understand-why-system-provided-decision,ai-system-evaluation.system-criteria-acceptable,newsitem.7.rating-after-xai,ai-system-evaluation.benefit-of-doubt,newsitem.8.rating-before-xai,ai-system-evaluation.system-decides-consistently,ai-system-evaluation.everyday-work-helpful,ai-system-evaluation.system-has-functionality-for-work,ai-system-evaluation.why-not-helpful-for-everyday-work,workDurationInSeconds.main
0,5,6,6,4,EGC87,6,6,2,4,6,...,5,4,95,3,95,4,,,,1943
1,7,7,1,7,,7,7,0,7,7,...,7,6,95,6,90,6,4,5,,1787
2,7,6,7,7,X,7,7,2,7,7,...,6,6,90,6,85,6,,,,543
3,5,5,3,5,X,5,4,2,4,4,...,4,4,97,6,46,1,,,,1785
4,6,4,4,4,FMC53,7,6,2,3,6,...,7,4,81,4,31,5,,,,1079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,4,3,4,4,JXT807,4,3,2,5,3,...,3,5,75,6,50,5,,,,1740
138,5,7,4,6,HDR20,6,7,2,3,7,...,5,2,91,5,48,2,,,,928
139,7,6,5,4,KAH15,7,5,2,5,6,...,6,6,87,6,84,5,,,,1353
140,4,4,6,4,HES99,5,6,2,3,6,...,6,6,64,4,72,5,,,,656


In [10]:
# dataframe of participants who did the qualification task for v1 and the main task for v2
pd.merge(df_v1_q, df_v2_m, on='swymerId', how='inner', suffixes=('.qualification', '.main'))

,information-literacy.identify-misinformation,information-seeking.sources.online-newspaper,expectations.sceptical-about-ai,information-seeking.sources.personal-social-networks,personal-code,information-literacy.efficiently-use-subscribed-and-openaccess,information-seeking.purpose.keep-up-to-date,POINTS.qualification,information-seeking.truthfulness.facebook,information-seeking.purpose.general-awareness,...,ai-system-evaluation.understand-why-system-provided-decision,ai-system-evaluation.system-criteria-acceptable,newsitem.7.rating-after-xai,ai-system-evaluation.benefit-of-doubt,newsitem.8.rating-before-xai,ai-system-evaluation.system-decides-consistently,ai-system-evaluation.everyday-work-helpful,ai-system-evaluation.system-has-functionality-for-work,ai-system-evaluation.why-not-helpful-for-everyday-work,workDurationInSeconds.main
0,7,6,7,7,X,7,7,2,7,7,...,7,1,91,6,85,1,,,NaN,493
1,7,6,5,5,X,6,6,0,6,7,...,7,3,82,5,92,4,4,2,NaN,558
2,6,4,3,6,LMB54,5,6,2,5,6,...,7,6,78,7,52,6,,,,2062
3,6,6,5,6,SDO8,6,6,2,5,7,...,4,3,90,3,55,3,,,,802
4,7,6,5,7,X,6,7,2,6,6,...,2,1,79,1,91,2,,,,970
5,3,1,2,6,Csr54,6,1,2,5,1,...,4,4,100,7,88,4,,,,423
6,5,7,4,6,YAU38,5,6,2,4,6,...,6,6,80,5,76,6,,,,1299
7,6,5,3,1,DRS48,6,6,2,4,6,...,7,7,85,3,22,7,,,,1198
8,5,4,3,4,ldi91,7,5,2,2,6,...,6,3,48,2,36,5,,,,889
9,6,7,4,7,nsp92,6,6,2,3,7,...,7,6,90,6,70,7,,,,2805


In [11]:
# dataframe of participants who are in the main task of v2, but not in either qualification task
df_v2_m[~df_v2_m.swymerId.isin(df_v1_q.swymerId) & ~df_v2_m.swymerId.isin(df_v2_q.swymerId)]

,news-dashboard-evaluation.assist-decisions,ai-system-evaluation.most-useful-explanation-feature,newsitem.7.system-evaluation.understand-what-system-does,ai-system-evaluation.easy-to-follow-what-system-does,newsitem.11.system-evaluation.classified-correctly,ai-system-evaluation.tend-to-trust-system-even-without-knowledge,news-dashboard-evaluation.useful-assess-news-articles,newsitem.8.system-evaluation.classified-correctly,newsitem.11.system-evaluation.explanations-comprehensible-and-help-assess,newsitem.8.system-evaluation.explanations-comprehensible-and-help-assess,...,ai-system-evaluation.system-criteria-acceptable,newsitem.7.rating-after-xai,ai-system-evaluation.benefit-of-doubt,newsitem.8.rating-before-xai,ai-system-evaluation.system-decides-consistently,ai-system-evaluation.everyday-work-helpful,ai-system-evaluation.system-has-functionality-for-work,ai-system-evaluation.why-not-helpful-for-everyday-work,swymerId,workDurationInSeconds
1,2,readability,5,4,2,2,5,4,3,4,...,2,30,3,52,1,2,3,Yes,CX_761234431695,276
59,3,truthfulness-assessment,5,3,2,4,3,3,5,6,...,3,64,5,55,4,5,4,,CX_961238893699,291
99,3,readability,5,3,6,2,3,5,6,5,...,4,84,5,83,4,,,,CX_381241034216,682
117,3,truthfulness-assessment,3,3,5,1,4,2,1,2,...,2,81,3,49,3,,,,CX_861245307212,2978
0,7,truthfulness-assessment,6,7,6,6,6,6,6,6,...,6,90,6,34,6,5,5,NaN,CX_431259207049,1386
39,5,publishing-date,4,5,7,5,5,7,5,5,...,5,90,6,35,4,6,5,NaN,CX_931183560242,1731


In [12]:
# merge df_v2_q and df_v2_m on swymerId, 
# only keep rows where aliasId is in both dataframes (inner join)
# keep all columns from both and suffix them with .qualification and .main (e.g. for POINTS)
df_v2 = pd.merge(df_v2_q, df_v2_m, on='swymerId', how='inner', suffixes=('.qualification', '.main'))
df_v2

,information-literacy.identify-misinformation,information-seeking.sources.online-newspaper,expectations.sceptical-about-ai,information-seeking.sources.personal-social-networks,personal-code,information-literacy.efficiently-use-subscribed-and-openaccess,information-seeking.purpose.keep-up-to-date,POINTS.qualification,information-seeking.truthfulness.facebook,information-seeking.purpose.general-awareness,...,ai-system-evaluation.understand-why-system-provided-decision,ai-system-evaluation.system-criteria-acceptable,newsitem.7.rating-after-xai,ai-system-evaluation.benefit-of-doubt,newsitem.8.rating-before-xai,ai-system-evaluation.system-decides-consistently,ai-system-evaluation.everyday-work-helpful,ai-system-evaluation.system-has-functionality-for-work,ai-system-evaluation.why-not-helpful-for-everyday-work,workDurationInSeconds.main
0,6,6,4,7,DSF46,4,7,2,4,7,...,5,6,90,5,44,6,,,,1090
1,6,2,6,4,HGD03,6,6,2,4,6,...,6,5,78,5,21,6,,,,3154
2,6,6,4,1,psl46,7,6,2,3,6,...,7,7,26,5,10,5,,,,1380
3,5,5,4,2,RJL06,6,5,2,2,5,...,5,5,61,6,79,3,,,,1457
4,5,5,2,2,ADN44,6,5,2,5,6,...,6,6,85,6,30,6,,,,1388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,6,3,5,6,ABA69,6,7,2,3,7,...,7,6,97,6,46,6,,,NaN,1066
141,4,3,2,1,LMI98,1,6,2,5,4,...,6,6,85,5,50,5,,,NaN,881
142,4,4,5,2,eat33,4,5,2,3,5,...,6,5,88,5,65,5,,,NaN,1782
143,6,7,5,1,SER56,7,7,2,1,6,...,3,6,78,6,78,6,,,NaN,997


In [13]:
df_v3 = pd.merge(df_v3_q, df_v3_m, on='swymerId', how='inner', suffixes=('.qualification', '.main'))
df_v3

,information-literacy.identify-misinformation,information-seeking.sources.online-newspaper,expectations.sceptical-about-ai,information-seeking.sources.personal-social-networks,personal-code,information-literacy.efficiently-use-subscribed-and-openaccess,newsitem.10.system-evaluation.understand-what-system-does,information-seeking.purpose.keep-up-to-date,newsitem.10.system-evaluation.xai-features-useful,POINTS.qualification,...,ai-system-evaluation.understand-why-system-provided-decision,ai-system-evaluation.system-criteria-acceptable,newsitem.7.rating-after-xai,ai-system-evaluation.benefit-of-doubt,newsitem.8.rating-before-xai,ai-system-evaluation.system-decides-consistently,ai-system-evaluation.everyday-work-helpful,ai-system-evaluation.system-has-functionality-for-work,ai-system-evaluation.why-not-helpful-for-everyday-work,workDurationInSeconds.main
0,1,2,2,1,X,1,,2,,0,...,7,7,91,7,92,5,7,7,,1084
1,4,4,5,5,gan61,5,6,7,6,2,...,6,6,95,5,22,5,,,,1204
2,5,4,5,3,MEM63,5,6,4,6,2,...,5,6,80,5,55,4,,,,748
3,6,6,5,2,CJD01,7,5,7,7,2,...,6,6,90,6,34,4,,,,800
4,5,5,5,1,kjm19,7,7,6,7,2,...,5,6,93,6,69,6,,,,1131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,7,7,3,6,Kap14,7,7,7,6,2,...,6,7,48,7,47,5,,,,682
137,6,5,5,4,JPW92,6,7,6,7,2,...,3,3,80,6,65,3,,,,793
138,1,7,2,7,mdp43,1,7,5,7,2,...,7,7,87,7,65,7,,,,1209
139,6,5,4,4,pdb43,6,7,7,7,2,...,6,4,42,4,91,5,,,,733


In [14]:
df = pd.concat([df_v1, df_v2, df_v3], axis=0, ignore_index=True)
df["is-journalist"] = False
df["POINTS"] = df["POINTS.main"].astype(int) + df["POINTS.qualification"].astype(int)
df["workDurationInSeconds"] = df["workDurationInSeconds.main"] + df["workDurationInSeconds.qualification"]

df_j["is-journalist"] = True

df = pd.concat([df, df_j], axis=0, ignore_index=True)

C:\Users\Csomor\AppData\Local\Temp\ipykernel_11912\1237084652.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["is-journalist"] = False
C:\Users\Csomor\AppData\Local\Temp\ipykernel_11912\1237084652.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["POINTS"] = df["POINTS.main"].astype(int) + df["POINTS.qualification"].astype(int)
C:\Users\Csomor\AppData\Local\Temp\ipykernel_11912\1237084652.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which 

In [15]:
columns_to_rename = {
  "swymerId": "METADATA.swymerId",
  "workDurationInSeconds.main": "METADATA.workDurationInSeconds.main",
  "workDurationInSeconds.qualification": "METADATA.workDurationInSeconds.qualification",
  "workDurationInSeconds": "METADATA.workDurationInSeconds",
  "personal-code": "METADATA.personal-code",
  "is-journalist": "JOURNALIST",
  "METADATA.FEATURE": "FEATURE",
}

df = df.rename(columns=columns_to_rename)

In [16]:
# order columns alphabetically
df = df.reindex(sorted(df.columns), axis=1)

In [17]:
from pprint import pprint

columns = list(df.columns)
pprint(columns)

['FEATURE',
 'JOURNALIST',
 'METADATA.GROUP',
 'METADATA.personal-code',
 'METADATA.swymerId',
 'METADATA.workDurationInSeconds',
 'METADATA.workDurationInSeconds.main',
 'METADATA.workDurationInSeconds.qualification',
 'POINTS',
 'POINTS.main',
 'POINTS.qualification',
 'ai-system-evaluation.additional-functionality',
 'ai-system-evaluation.benefit-of-doubt',
 'ai-system-evaluation.classification-comprehensible',
 'ai-system-evaluation.criteria-to-judge-reliability',
 'ai-system-evaluation.easy-to-follow-what-system-does',
 'ai-system-evaluation.everyday-work-helpful',
 'ai-system-evaluation.know-what-will-happen-next-time',
 'ai-system-evaluation.most-useful-explanation-feature',
 'ai-system-evaluation.other-information-wish',
 'ai-system-evaluation.rely-on-ai-system',
 'ai-system-evaluation.sceptical-about-ai-system',
 'ai-system-evaluation.system-able-to-classify-news-articles',
 'ai-system-evaluation.system-able-to-detect-fake-news',
 'ai-system-evaluation.system-can-correctly-cla

In [18]:
# determine which columns only contain values which can be parsed to numbers
def is_number(s):
    if s == "":
        return True # empty cells are treated as possible numbers
    try:
        float(s)
        return True
    except ValueError:
        return False

In [19]:
# determine which columns only contain values which can be parsed to integers or are empty
# change data type of numeric columns to int
numeric_columns = df.columns[df.applymap(is_number).all()]
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, downcast='integer')

In [20]:
rating_columns = [col for col in df.columns if ".rating-" in col]

In [21]:
# replace all value in rating columns which are < 0 with 0
df[rating_columns] = df[rating_columns].mask(df[rating_columns] < 0, 0)

# replace all value in rating columns which are > 100 with 100
df[rating_columns] = df[rating_columns].mask(df[rating_columns] > 100, 100)

In [22]:
df.to_csv("data/data_cleaned.csv", index=False)